In [1]:
import os
import math
import random

import numpy as np
import tensorflow as tf
import cv2

slim =  tf.contrib.slim

D:\python\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
import sys
sys.path.append('../')

In [ ]:
from model import configuration, inception, crnn
from data_utils import vocabulary

In [ ]:
image_dir = './data/allSubUni'
dataset_dir = './data/output/'
file_pattern = 'ocr10_train_*.tfrecord'
weights_path='./log/model.ckpt-29001'

In [ ]:
img_input = tf.placeholder(tf.uint8, shape=(32, None, 3))
image = tf.to_float(img_input)
image = tf.expand_dims(image, 0)
vocab = vocabulary.Vocabulary()
vocab.build()
model_config = configuration.ModelConfig()
model_config

training_config = configuration.TrainingConfig()

model = crnn.CRNN(
            vocab,
            model_config,
            mode="inference",
        )
with tf.variable_scope('CRNN', reuse=False):
    image = model.build_image_embeddings(image)
    net_out, _ = model.build_model(image)
    
decodes, _ = tf.nn.ctc_beam_search_decoder(inputs=net_out, sequence_length=25*np.ones(1), merge_repeated=False)
pred = tf.sparse_tensor_to_dense(decodes[0])

In [ ]:
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [ ]:
image_path = './data/sim_sub_15w/0_0_27HJF30FRY.jpg'
#image_path = './data/allSubUni/1612ELL127323C00155.jpg'
#image_path = './test_01.jpg'
with sess.as_default():
        
    inputdata = cv2.imread(image_path, cv2.IMREAD_COLOR)
    #scale = 32.0 / inputdata.shape[0]
    #width = max(0 , scale * inputdata.shape[1])
    inputdata = cv2.resize(inputdata, (100, 32))
    
    saver.restore(sess=sess, save_path=weights_path)

    out = sess.run(pred, feed_dict={img_input: inputdata})
    #preds = decoder.writer.sparse_tensor_to_str(preds[0])
    print(vocab._to_string(out[0]))
    plt.figure(1)
    plt.imshow(cv2.imread(image_path, cv2.IMREAD_COLOR)[:, :, (2, 1, 0)])
    plt.figure(2)
    plt.imshow(inputdata)
    plt.show()

In [5]:
test='jpeg'

In [7]:
test.encode('utf-8')

b'jpeg'

In [8]:
import glob

In [12]:
ddir = './data/sim_sub_15w/*.jpg'

In [14]:
test = glob.glob(ddir)

In [15]:
test[0]

'./data/sim_sub_15w\\0_0_00L61IZ4SA.jpg'

In [ ]:
vocab = vocabulary.Vocabulary()
vocab.build()
with tf.device('/cpu:0'):
    model_config = configuration.ModelConfig()
    model_config.batch_size = 1
    model_config.dataset_dir = dataset_dir
    model_config.input_file_pattern = file_pattern

    training_config = configuration.TrainingConfig()

    model = crnn.CRNN(
        vocab,
        model_config,
        mode="train",
    )
    images, input_labels = model.build_inputs()
    input_labels = tf.sparse_tensor_to_dense(input_labels)
#sess = tf.Session()
#sess.run(tf.global_variables_initializer())

In [ ]:
with tf.Session() as sess:
    with slim.queues.QueueRunners(sess):
        for i in range(1):
            im, lb = sess.run([images, input_labels])
            print(im.shape)
            im = np.uint8(im[0,:,:,:])
            plt.imshow(im)
            plt.show()
            print(lb)